In [2]:
from google.colab import drive
drive.mount("/content/drive")

!pip install datasets evaluate transformers[sentencepiece]

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 34.5 MB/s 
     |████████████████████████████████| 72 kB 1.5 MB/s 
     |████████████████████████████████| 5.5 MB 53.3 MB/s 
     |████████████████████████████████| 163 kB 71.6 MB/s 
     |████████████████████████████████| 115 kB 71.4 MB/s 
     |████████████████████████████████| 95 kB 4.9 MB/s 
     |████████████████████████████████| 212 kB 65.8 MB/s 
     |████████████████████████████████| 127 kB 68.4 MB/s 
     |████████████████████████████████| 115 kB 69.7 MB/s 
     |████████████████████████████████| 7.6 MB 57.3 MB/s 
     |████████████████████████████████| 1.3 MB 59.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Unins

Image classification assigns a label or class to an image. Unlike text or audio classification, the inputs are the pixel values that represent an image. There are many uses for image classification, like detecting damage after a disaster, monitoring crop health, or helping screen medical images for signs of disease.

This guide will show you how to fine-tune ViT on the Food-101 dataset to classify a food item in an image.

#  Load Food-101 dataset

Load only the first 5000 images of the Food-101 dataset from the 🤗 Datasets library since it is pretty large:

In [3]:
from datasets import load_dataset

food = load_dataset("food101", split="train[:5000]")

Generating train split:   0%|          | 0/75750 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/25250 [00:00<?, ? examples/s]

Dataset food101 downloaded and prepared to /root/.cache/huggingface/datasets/food101/default/0.0.0/7cebe41a80fb2da3f08fcbef769c8874073a86346f7fb96dc0847d4dfc318295. Subsequent calls will reuse this data.


Split this dataset into a train and test set:

In [4]:
food = food.train_test_split(test_size=0.2)

Then take a look at an example:

In [5]:
food["train"][0]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x384 at 0x7F47ECA6B790>,
 'label': 79}

The image field contains a PIL image, and each label is an integer that represents a class. Create a dictionary that maps a label name to an integer and vice versa. The mapping will help the model recover the label name from the label number:

In [6]:
labels = food["train"].features["label"].names
label2id, id2label = dict(), dict() 
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label 

Now you can convert the label number to a label name for more information:

In [7]:
id2label[str(79)]

'prime_rib'

Each food class - or label - corresponds to a number; 79 indicates a prime rib in the example above.

# Preprocess

Load the ViT feature extractor to process the image into a tensor:

In [8]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")

Downloading:   0%|          | 0.00/160 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/502 [00:00<?, ?B/s]

Apply several image transformations to the dataset to make the model more robust against overfitting. Here you’ll use torchvision’s transforms module. Crop a random part of the image, resize it, and normalize it with the image mean and standard deviation:

In [9]:
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

normalize = Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
_transforms = Compose([RandomResizedCrop(feature_extractor.size), ToTensor(), normalize])

Create a preprocessing function that will apply the transforms and return the pixel_values - the inputs to the model - of the image:

In [10]:
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples

Use 🤗 Dataset’s with_transform method to apply the transforms over the entire dataset. The transforms are applied on-the-fly when you load an element of the dataset:

In [11]:
food = food.with_transform(transforms)

In [12]:
food["train"][0]

{'label': 79,
 'pixel_values': tensor([[[-0.2706, -0.2235, -0.1843,  ...,  0.3020,  0.2235,  0.2235],
          [-0.2706, -0.2392, -0.2235,  ...,  0.6000,  0.5529,  0.4667],
          [-0.2941, -0.2627, -0.2549,  ...,  0.7176,  0.7098,  0.6706],
          ...,
          [ 0.8510,  0.8588,  0.8824,  ...,  0.9373,  0.9529,  0.9529],
          [ 0.8118,  0.8353,  0.8745,  ...,  0.9451,  0.9529,  0.9294],
          [ 0.8118,  0.8510,  0.8824,  ...,  0.9373,  0.9294,  0.9373]],
 
         [[-0.4510, -0.4118, -0.3647,  ..., -0.5686, -0.5686, -0.5059],
          [-0.4431, -0.4196, -0.4196,  ..., -0.5529, -0.5765, -0.5922],
          [-0.4431, -0.4431, -0.4588,  ..., -0.5529, -0.5451, -0.5608],
          ...,
          [ 0.7098,  0.7098,  0.7490,  ...,  0.9137,  0.9059,  0.9059],
          [ 0.6863,  0.7020,  0.7412,  ...,  0.9216,  0.9216,  0.9059],
          [ 0.6863,  0.7176,  0.7490,  ...,  0.9059,  0.9059,  0.9137]],
 
         [[-0.6863, -0.6392, -0.6000,  ..., -0.7412, -0.7020, -0.6471]

Use DefaultDataCollator to create a batch of examples. Unlike other data collators in 🤗 Transformers, the DefaultDataCollator does not apply additional preprocessing such as padding.

In [13]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

#  Train

Load ViT with AutoModelForImageClassification. Specify the number of labels, and pass the model the mapping between label number and label class:

In [14]:
from transformers import AutoModelForImageClassification, TrainingArguments

model = AutoModelForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
)

Downloading:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


At this point, only three steps remain:

1. Define your training hyperparameters in TrainingArguments. It is important you don’t remove unused columns because this will drop the image column. Without the image column, you can’t create pixel_values. Set remove_unused_columns=False to prevent this behavior!
2. Pass the training arguments to Trainer along with the model, datasets, tokenizer, and data collator.
3. Call train() to fine-tune your model.

In [15]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/HuggingFace/model_files/image_classification/google-vit/results",
    per_device_train_batch_size=16,
    evaluation_strategy="steps",
    num_train_epochs=4,
    fp16=True,
    save_steps=100,
    eval_steps=100,
    logging_steps=10,
    learning_rate=2e-4,
    save_total_limit=2,
    remove_unused_columns=False,
)

In [16]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=food["train"],
    eval_dataset=food["test"],
    tokenizer=feature_extractor,
)

Using cuda_amp half precision backend


In [17]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 4000
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1000
  Number of trainable parameters = 85876325


Step,Training Loss,Validation Loss
100,0.795600,0.774930
200,0.533000,0.511606
300,0.394100,0.581958
400,0.322400,0.509354
500,0.318900,0.368216
600,0.287000,0.335073
700,0.150000,0.316066
800,0.231100,0.344351
900,0.151400,0.288565
1000,0.194600,0.294274


***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/HuggingFace/model_files/image_classification/google-vit/results/checkpoint-100
Configuration saved in /content/drive/MyDrive/HuggingFace/model_files/image_classification/google-vit/results/checkpoint-100/config.json
Model weights saved in /content/drive/MyDrive/HuggingFace/model_files/image_classification/google-vit/results/checkpoint-100/pytorch_model.bin
Feature extractor saved in /content/drive/MyDrive/HuggingFace/model_files/image_classification/google-vit/results/checkpoint-100/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/HuggingFace/model_files/image_classification/google-vit/results/checkpoint-200
Configuration saved in /content/drive/MyDrive/HuggingFace/model_files/image_classification/google-vit/results/checkpoint-200/config.json
Model weights saved in /content

TrainOutput(global_step=1000, training_loss=0.44839981412887575, metrics={'train_runtime': 482.445, 'train_samples_per_second': 33.164, 'train_steps_per_second': 2.073, 'total_flos': 1.2409719791616e+18, 'train_loss': 0.44839981412887575, 'epoch': 4.0})

In [20]:
train_dataset[0]

NameError: ignored

In [24]:
preds = trainer.predict(test_dataset=food["test"])

***** Running Prediction *****
  Num examples = 1000
  Batch size = 8


In [26]:
preds

PredictionOutput(predictions=array([[-2.188, -2.3  , -2.393, ..., -2.494, -2.541, -2.39 ],
       [-2.197, -2.332, -2.24 , ..., -2.19 , -2.29 , -2.332],
       [-2.88 , -3.002, -2.922, ..., -3.002, -3.049, -2.875],
       ...,
       [-2.07 , -2.178, -2.229, ..., -2.26 , -2.354, -2.02 ],
       [-3.334, -3.26 , -3.488, ..., -3.525, -3.514, -3.172],
       [-2.67 , -2.666, -2.67 , ..., -2.732, -2.7  , -2.633]],
      dtype=float16), label_ids=array([53, 20, 81, 20, 53,  6, 79, 20, 79, 81, 77, 10, 81, 79, 10, 20, 77,
       53, 77, 79, 81, 79, 81, 77, 79, 53, 79,  6, 10, 79,  6, 10, 77, 81,
       79, 20, 20,  6, 20, 10, 79, 77, 10, 20, 77, 20, 81, 10,  6, 10, 77,
       10, 10, 53, 53,  6,  6,  6, 20, 79, 79, 53, 81, 79, 79, 10, 20,  6,
       77, 20,  6, 81, 53, 53, 79, 20, 10, 53, 10, 79, 20, 77, 10, 79, 53,
       20, 77, 20,  6, 79,  6, 20, 20, 20, 77, 79, 10, 81, 79, 81, 10, 20,
        6, 81, 77, 77, 53, 20, 53,  6, 81, 77, 81, 77,  6, 20, 81,  6,  6,
       79, 53, 79, 53, 81,  6

In [27]:
preds[1]

array([53, 20, 81, 20, 53,  6, 79, 20, 79, 81, 77, 10, 81, 79, 10, 20, 77,
       53, 77, 79, 81, 79, 81, 77, 79, 53, 79,  6, 10, 79,  6, 10, 77, 81,
       79, 20, 20,  6, 20, 10, 79, 77, 10, 20, 77, 20, 81, 10,  6, 10, 77,
       10, 10, 53, 53,  6,  6,  6, 20, 79, 79, 53, 81, 79, 79, 10, 20,  6,
       77, 20,  6, 81, 53, 53, 79, 20, 10, 53, 10, 79, 20, 77, 10, 79, 53,
       20, 77, 20,  6, 79,  6, 20, 20, 20, 77, 79, 10, 81, 79, 81, 10, 20,
        6, 81, 77, 77, 53, 20, 53,  6, 81, 77, 81, 77,  6, 20, 81,  6,  6,
       79, 53, 79, 53, 81,  6, 81, 79, 81, 20, 77,  6,  6, 81, 10, 10, 20,
       77, 79, 53, 81, 77, 10, 10, 81,  6, 20, 79, 77, 81, 20, 20, 10, 53,
       10,  6, 77, 81, 79, 81, 53, 10, 20, 79, 81, 81, 10, 81, 10, 20, 53,
       20, 79, 81,  6, 10, 81, 77, 53, 79, 10,  6, 53, 81, 53, 79, 81, 10,
       20, 20, 20, 20,  6, 53, 20, 10,  6, 81, 20, 10, 20,  6, 79, 10, 10,
       53, 79,  6, 53,  6, 20, 77, 20, 79,  6,  6, 53, 10,  6,  6, 10, 10,
       53,  6, 79, 53, 20

In [28]:
preds[1][0]

53

In [29]:
food["test"][0]

{'label': 53,
 'pixel_values': tensor([[[-0.3098, -0.4667, -0.4902,  ...,  0.0745,  0.0667,  0.1294],
          [-0.3647, -0.4824, -0.4824,  ...,  0.0353,  0.1529,  0.1843],
          [-0.3569, -0.4275, -0.4745,  ...,  0.0196,  0.2314,  0.2471],
          ...,
          [ 0.1137,  0.1137,  0.1137,  ...,  0.3255,  0.2941,  0.2784],
          [ 0.1137,  0.1059,  0.1137,  ...,  0.3412,  0.3255,  0.2706],
          [ 0.0431,  0.0510,  0.0902,  ...,  0.3569,  0.3412,  0.2784]],
 
         [[-0.4118, -0.5608, -0.5765,  ..., -0.3098, -0.3176, -0.2392],
          [-0.4588, -0.5765, -0.5608,  ..., -0.3412, -0.2314, -0.1843],
          [-0.4510, -0.5216, -0.5529,  ..., -0.3647, -0.1529, -0.1216],
          ...,
          [ 0.3647,  0.3569,  0.3412,  ...,  0.1373,  0.1059,  0.1059],
          [ 0.3961,  0.3804,  0.3725,  ...,  0.1686,  0.1373,  0.1137],
          [ 0.3490,  0.3490,  0.3569,  ...,  0.1922,  0.1686,  0.1137]],
 
         [[-0.2941, -0.4510, -0.4824,  ..., -0.5765, -0.5765, -0.4588]